In [ ]:
import torch
import sys
import glob
import os
from tqdm.auto import tqdm, trange
sys.path.append("..")
sys.path.append("../..")
import matplotlib.pyplot as plt
import plotly.io as pio
import umap
from torch.utils import benchmark
from hist_optimizer import *

from ray_tools.base.transform import MultiLayer
from ray_tools.base.engine import RayEngine
from ray_nn.data.lightning_data_module import DefaultDataModule
from ray_nn.nn.xy_hist_data_models import MetrixXYHistSurrogate, StandardizeXYHist, HistSurrogateEngine, Model
from ray_tools.base.backend import RayBackendDockerRAYUI
from ray_tools.simulation.torch_datasets import BalancedMemoryDataset, MemoryDataset, HistDataset
from ray_optim.plot import Plot
from ray_tools.base.transform import Histogram, RayTransformConcat, XYHistogram
from ray_nn.data.transform import Select
from ray_tools.base.parameter import NumericalParameter, OutputParameter, NumericalOutputParameter, MutableParameter, RayParameterContainer
from sub_projects.ray_optimization.real_data import import_data
from sub_projects.ray_optimization.utils import ray_dict_to_tensor, ray_output_to_tensor

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Model

In [ ]:
root_dir = '../../'
engine = RayEngine(rml_basefile=os.path.join(root_dir, 'rml_src/METRIX_U41_G1_H1_318eV_PS_MLearn_1.15.rml'),
                                exported_planes=["ImagePlane"],
                                ray_backend=RayBackendDockerRAYUI(docker_image='ray-ui-service',
                                                                  docker_container_name='ray-ui-service-test',
                                                                  dockerfile_path='../../ray_docker/rayui',
                                                                  ray_workdir='/dev/shm/ray-workdir',
                                                                  verbose=False),
                                num_workers=-1,
                                as_generator=False)


model_path = os.path.join(root_dir, "outputs/xy_hist/ft1rr9h0/checkpoints/epoch=68-step=65665644.ckpt")
surrogate_engine = HistSurrogateEngine(checkpoint_path=model_path)

model = Model(path=model_path)

In [ ]:
%matplotlib widget
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt

def f(X, Y, Z):
    inp = torch.ones((1,model.mutable_parameter_count), device=model.device)*0.5
    inp[:,-3] = X
    inp[:,-2] = Y
    inp[:,-4] = Z
    ##out = model(inp)
    out = surrogate_engine.run([tensor_to_param_container(inp[0], model.input_parameter_container)])
    #out =  engine.run([tensor_to_param_container(inp[0], model.input_parameter_container)])
    #out = [XYHistogram(50, (-10., 10.), (-3., 3.))(out[0]['ray_output']['ImagePlane'])['histogram'].flatten()]
    out = [torch.cat((out[0]['ray_output']['ImagePlane']['xy_hist'].x_loc, out[0]['ray_output']['ImagePlane']['xy_hist'].y_loc))]
    return out[0].detach().cpu().numpy()#A*x**2 + B*x + Z

#fig = plt.figure()
fig, ax = plt.subplots(1, 2)
histogram_lims = model.histogram_lims
lineX, = ax[0].plot(np.linspace(histogram_lims[0][0], histogram_lims[0][1], num=50), f(X=.5, Y=.5, Z=.0)[:50])
lineY, = ax[1].plot(np.linspace(histogram_lims[1][0], histogram_lims[1][1], num=50), f(X=.5, Y=.5, Z=.0)[50:])

def update(X = .5, Y = .5, Z = .0):
    new_f = f(X,Y,Z)
    new_x = new_f[:50]
    lineX.set_ydata(new_x)
    ax[0].set_ylim(new_x.min(), new_x.max())
    new_y = new_f[50:]
    lineY.set_ydata(new_y)
    ax[1].set_ylim(new_y.min(), new_y.max())
    fig.canvas.draw_idle()
    
interact(update, X = (0,1,0.05), Y = (0,1,0.05), Z = (0,1,0.1));

In [ ]:
inp_list = []
for i in range(20):
    inp = torch.ones(model.mutable_parameter_count, device=model.device)*0.5
    #inp[-3] = 0.5# + i*0.5
    inp[-4] = 0. + i*0.05
    inp_list.append(inp)

#Plot.plot_engines_comparison(engine, surrogate_engine, [tensor_to_param_container(inp, model.input_parameter_container) for inp in inp_list], MultiLayer([0.]))
inp = torch.stack(inp_list)
plot_param_tensors( inp, inp, engine = engine, ray_parameter_container=model.input_parameter_container, compensated_parameters = inp)

In [ ]:
out = model(inp)
plotted, _ = out.max(dim=-1)
plt.plot(plotted.cpu())

# Look with model for new sample

In [ ]:
offsets_selected, uncompensated_parameters_selected, compensated_parameters_selected = find_good_offset_problem(model, fixed_parameters = [8, 14, 20, 21, 27, 28, 34])

with torch.no_grad():
    observed_rays = model(compensated_parameters_selected)

In [ ]:
offsets_list = []
uncompensated_parameters_list = []
compensated_parameters_list = []
for i in range(5000):
    offsets_selected, uncompensated_parameters_selected, compensated_parameters_selected = find_good_offset_problem(model, fixed_parameters = [8, 14, 20, 21, 27, 28, 34])
    offsets_list.append(offsets_selected)
    uncompensated_parameters_list.append(uncompensated_parameters_selected)
    compensated_parameters_list.append(compensated_parameters_selected)

offsets_list = torch.stack(offsets_list)
print(offsets_selected.mean().item(), "±", offsets_selected.std().item())

In [ ]:
import matplotlib.pyplot as plt
data = uncompensated_parameters_selected[:, 0, 0, :]
fig, ax = plt.subplots(2,1, figsize=(32,8))

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Plot each line with a different color from the cycle
for i, y in enumerate(uncompensated_parameters_list):
    ax[0].plot(y[:, 10, 0].T.cpu(), color=colors[i % len(colors)], alpha=0.2)
    ax[0].set_title("Uncompensated parameters")
    ax[0].set_xlabel("Beamline parameter [#]")
    ax[0].set_xticks(torch.arange(37))
    ax[1].plot(compensated_parameters_list[i][:, 10, 0].T.cpu(), color=colors[i % len(colors)], alpha=0.2)
    ax[1].set_title("Compensated parameters")
    ax[1].set_xlabel("Beamline parameter [#]")
    ax[1].set_xticks(torch.arange(37))
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True)
for entry in offsets_list:
    ax[0].scatter(entry[0].detach().cpu(), y=torch.arange(37).detach().cpu())
    ax[0].set_ylabel("Beamline parameter [#]")
    ax[0].set_xlabel("Chosen offset")
    ax[0].set_title("Simulated real beamline")
for entry in offsets_list:
    ax[1].scatter(torch.rand(37).detach().cpu(), y=torch.arange(37).detach().cpu())
    ax[1].set_xlabel("Chosen offset")
    ax[1].set_title("Random")

In [ ]:
from scipy.stats import kstest

def check_if_uniform(check_list):
    not_uniform_list = []
    for i, entry in enumerate(check_list.T):
        statistic, p_value = kstest(entry.cpu(), 'uniform', args=(0, 1))
        # Interpret p-value
        if p_value > 0.01:  # Common significance level is 0.05
            #print("Fail to reject the null hypothesis: Data appears to be uniformly distributed.")
            continue
        else:
            print(i, "Reject the null hypothesis: Data does not appear to be uniformly distributed.")
            not_uniform_list.append(i)
    return not_uniform_list

check_if_uniform(offsets_list.squeeze())

In [ ]:
mins, _ = uncompensated_parameters_selected[:, 0, 0, :].max(dim=0)
print((uncompensated_parameters_selected[:, 0, 0, :] == mins).sum(dim=0))

In [ ]:
for label, entry in model.offset_space.items():
    if isinstance(entry, MutableParameter):
        value_list = [value.value_lims for label2, value in model.input_parameter_container.items() if isinstance(value, MutableParameter) and label==label2]
        print("label:", entry.value_lims, "from", value_list[0])

In [ ]:
loss_min_params, loss, loss_min_list = optimize_smart_walker(model, observed_rays, uncompensated_parameters_selected, iterations=1000, num_candidates=10000, step_width=0.02)

In [ ]:
import torch
import numpy as np
from scipy.optimize import basinhopping

def torch_to_numpy(tensor):
    return tensor.detach().cpu().numpy()

def numpy_to_torch(array, device):
    return torch.tensor(array, device=device, dtype=torch.float32)

def objective_function(offsets_numpy, model, observed_rays, uncompensated_parameters):
    # Convert numpy array to torch tensor
    offsets = numpy_to_torch(offsets_numpy, device=model.device).unsqueeze(0)
    #offsets = torch.clamp(offsets, 0, 1)
    
    # Evaluate the model's loss with these offsets
    scaled_offsets = model.rescale_offset(offsets)
    compensated_rays = model(uncompensated_parameters + scaled_offsets)
    loss = ((compensated_rays - observed_rays) ** 2).mean().item()
    return loss

def optimize_smart_walker_scipy(model, observed_rays, uncompensated_parameters, iterations=100, hop_step=0.1, interval=1, stepsize=0.1):
    # Convert initial parameters to numpy for scipy
    initial_offsets = torch.rand(1, uncompensated_parameters.shape[-1], device=model.device)
    initial_offsets_numpy = torch_to_numpy(initial_offsets).flatten()

    # Define a list to track the minimum loss for each iteration
    loss_min_list = []

    # Set up the progress bar
    pbar = tqdm(total=iterations)

    def callback(x, f, accept):
        # Update progress bar and log the loss
        loss_min_list.append(f)
        pbar.update(1)
        pbar.set_postfix({"loss": f})

    # Configure basinhopping with scipy
    minimizer_kwargs = {
        "method": "Powell",
        "args": (model, observed_rays, uncompensated_parameters),
        "bounds": [(0, 1)] * initial_offsets_numpy.size  # Bounds for each parameter
    }
    
    # Run basinhopping
    result = basinhopping(
        objective_function, 
        initial_offsets_numpy, 
        niter=iterations,
        T=hop_step,
        stepsize=stepsize,
        interval=interval,
        minimizer_kwargs=minimizer_kwargs,
        callback=callback,
        disp=True
    )

    # Close the progress bar
    pbar.close()
    
    # Convert the final result back to torch for further use
    final_offsets_numpy = result.x
    final_offsets_torch = numpy_to_torch(final_offsets_numpy, device=model.device).view(1, -1)
    scaled_offsets = model.rescale_offset(final_offsets_torch.unsqueeze(1))
    best_loss_params = uncompensated_parameters + scaled_offsets

    return best_loss_params, result.fun, loss_min_list
loss_min_params, loss, loss_min_list = optimize_smart_walker_scipy(model, observed_rays, uncompensated_parameters_selected, hop_step=0.1, iterations=1000, interval=10, stepsize=.01)

In [ ]:
import optuna

optimize_tpe(model, observed_rays, uncompensated_parameters_selected, iterations=1000, num_candidates=None)

In [ ]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll_torch
from botorch.acquisition import qLogExpectedImprovement
from botorch.acquisition.monte_carlo import qNoisyExpectedImprovement
from botorch.optim import optimize_acqf
from botorch.utils.sampling import manual_seed
from botorch.utils.transforms import unnormalize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood

def objective_function(offsets, model, observed_rays, uncompensated_parameters):
    """
    Evaluate the objective function.
    """
    # Scale offsets and compute compensated rays
    scaled_offsets = model.rescale_offset(offsets)
    compensated_rays = model(uncompensated_parameters + scaled_offsets)

    # Calculate loss (mean squared error)
    loss = ((compensated_rays - observed_rays) ** 2).mean().item()
    return loss

def optimize_smart_walker_botorch(
    model, observed_rays, uncompensated_parameters, iterations=100, num_initial_points=20
):
    """
    Optimize the smart walker using BoTorch with TS/qEI.
    """
    device = model.device
    dtype = torch.float

    # Define parameter bounds
    bounds = torch.tensor([[0.0] * uncompensated_parameters.shape[-1], 
                           [1.0] * uncompensated_parameters.shape[-1]], device=device, dtype=dtype)
    dim = uncompensated_parameters.shape[-1]

    # Generate random initial samples
    train_X = torch.rand((num_initial_points, dim), device=device, dtype=dtype)
    train_Y = torch.tensor(
        [
            [objective_function(x, model, observed_rays, uncompensated_parameters)]
            for x in train_X
        ],
        device=device,
        dtype=dtype,
    )

    # Start optimization
    for i in range(iterations):
        # Fit a GP model
        gp = SingleTaskGP(train_X, train_Y)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_mll_torch(mll)

        # Define the acquisition function (qEI)
        qEI = qLogExpectedImprovement(model=gp, best_f=train_Y.min().item())

        # Optimize the acquisition function to find the next set of candidates
        candidates, _ = optimize_acqf(
            acq_function=qEI,
            bounds=bounds,
            q=1,  # Optimize one point at a time
            num_restarts=10,
            raw_samples=100,
        )

        # Evaluate the objective at the new candidate
        new_X = candidates.detach()
        new_Y = torch.tensor(
            [[objective_function(new_X[0], model, observed_rays, uncompensated_parameters)]],
            device=device,
            dtype=dtype,
        )

        # Append new data to the training set
        train_X = torch.cat([train_X, new_X], dim=0)
        train_Y = torch.cat([train_Y, new_Y], dim=0)

        # Print progress
        print(f"Iteration {i + 1}/{iterations}, Best Loss: {train_Y.min().item()}")

    # Return the best found parameters and their scaled offsets
    best_offset_index = train_Y.argmin()
    best_offsets = train_X[best_offset_index]
    scaled_offsets = model.rescale_offset(best_offsets.unsqueeze(0))
    loss_min_params = uncompensated_parameters + scaled_offsets
    best_loss = train_Y.min().item()

    return loss_min_params, best_loss, train_X, train_Y

# Example usage
loss_min_params, loss, train_X, train_Y = optimize_smart_walker_botorch(
    model, observed_rays, uncompensated_parameters_selected, iterations=100, num_initial_points=20
)

In [ ]:
#!/usr/bin/env python3
# coding: utf-8

# ## BO with TuRBO-1 and TS/qEI
# 
# In this tutorial, we show how to implement Trust Region Bayesian Optimization (TuRBO) [1] in a closed loop in BoTorch.
# 
# This implementation uses one trust region (TuRBO-1) and supports either parallel expected improvement (qEI) or Thompson sampling (TS). We optimize the $20D$ Ackley function on the domain $[-5, 10]^{20}$ and show that TuRBO-1 outperforms qEI as well as Sobol.
# 
# Since botorch assumes a maximization problem, we will attempt to maximize $-f(x)$ to achieve $\max_x -f(x)=0$.
# 
# [1]: [Eriksson, David, et al. Scalable global optimization via local Bayesian optimization. Advances in Neural Information Processing Systems. 2019](https://proceedings.neurips.cc/paper/2019/file/6c990b7aca7bc7058f5e98ea909e924b-Paper.pdf)
# 

# In[1]:


import os
import math
import warnings
from dataclasses import dataclass

import torch
from botorch.acquisition import qExpectedImprovement, qLogExpectedImprovement
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.fit import fit_gpytorch_mll
from botorch.generation import MaxPosteriorSampling
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf
from botorch.test_functions import Ackley
from botorch.utils.transforms import unnormalize
from torch.quasirandom import SobolEngine

import gpytorch
from gpytorch.constraints import Interval
from gpytorch.kernels import MaternKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood


warnings.filterwarnings("ignore", category=BadInitialCandidatesWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
SMOKE_TEST = os.environ.get("SMOKE_TEST")


# ## Optimize the 20-dimensional Ackley function
# 
# The goal is to minimize the popular Ackley function:
# 
# $f(x_1,\ldots,x_d) = -20\exp\left(-0.2 \sqrt{\frac{1}{d} \sum_{j=1}^d x_j^2} \right) -\exp \left( \frac{1}{d} \sum_{j=1}^d \cos(2 \pi x_j) \right) + 20 + e$
# 
# over the domain  $[-5, 10]^{20}$.  The global optimal value of $0$ is attained at $x_1 = \ldots = x_d = 0$.
# 
# As mentioned above, since botorch assumes a maximization problem, we instead maximize $-f(x)$.

# In[2]:

def objective_function(offsets, model, observed_rays, uncompensated_parameters):
    """
    Evaluate the objective function.
    """
    # Scale offsets and compute compensated rays
    scaled_offsets = model.rescale_offset(offsets)
    compensated_rays = model(uncompensated_parameters + scaled_offsets)

    # Calculate loss (mean squared error)
    loss = ((compensated_rays - observed_rays) ** 2).mean()
    return loss

def fun(offsets):
    return -objective_function(offsets.float(), model, observed_rays, uncompensated_parameters_selected).double()
    

#fun = Ackley(dim=20, negate=True).to(dtype=dtype, device=device)
#fun.bounds[0, :].fill_(0)
#fun.bounds[1, :].fill_(1)
dim = 37#fun.dim
lb, ub = (0., 1.) #fun.bounds

batch_size = 100
n_init = 2 * dim
max_cholesky_size = float("inf")  # Always use Cholesky


def eval_objective(x):
    """This is a helper function we use to unnormalize and evalaute a point"""
    return fun(x) #fun(unnormalize(x, fun.bounds))


# ## Maintain the TuRBO state
# TuRBO needs to maintain a state, which includes the length of the trust region, success and failure counters, success and failure tolerance, etc. 
# 
# In this tutorial we store the state in a dataclass and update the state of TuRBO after each batch evaluation. 
# 
# **Note**: These settings assume that the domain has been scaled to $[0, 1]^d$ and that the same batch size is used for each iteration.

# In[3]:


@dataclass
class TurboState:
    dim: int
    batch_size: int
    length: float = 0.8
    length_min: float = 0.5**7
    length_max: float = 1.6
    failure_counter: int = 0
    failure_tolerance: int = float("nan")  # Note: Post-initialized
    success_counter: int = 0
    success_tolerance: int = 10  # Note: The original paper uses 3
    best_value: float = -float("inf")
    restart_triggered: bool = False

    def __post_init__(self):
        self.failure_tolerance = math.ceil(
            max([4.0 / self.batch_size, float(self.dim) / self.batch_size])
        )


def update_state(state, Y_next):
    if max(Y_next) > state.best_value + 1e-3 * math.fabs(state.best_value):
        state.success_counter += 1
        state.failure_counter = 0
    else:
        state.success_counter = 0
        state.failure_counter += 1

    if state.success_counter == state.success_tolerance:  # Expand trust region
        state.length = min(2.0 * state.length, state.length_max)
        state.success_counter = 0
    elif state.failure_counter == state.failure_tolerance:  # Shrink trust region
        state.length /= 2.0
        state.failure_counter = 0

    state.best_value = max(state.best_value, max(Y_next).item())
    if state.length < state.length_min:
        state.restart_triggered = True
    return state


# ## Take a look at the state

# In[4]:


state = TurboState(dim=dim, batch_size=batch_size)
print(state)


# ## Generate initial points
# This generates an initial set of Sobol points that we use to start of the BO loop.

# In[5]:


def get_initial_points(dim, n_pts, seed=0):
    sobol = SobolEngine(dimension=dim, scramble=True, seed=seed)
    X_init = sobol.draw(n=n_pts).to(dtype=dtype, device=device)
    return X_init


# ## Generate new batch
# Given the current `state` and a probabilistic (GP) `gp_model` built from observations `X` and `Y`, we generate a new batch of points.  
# 
# This method works on the domain $[0, 1]^d$, so make sure to not pass in observations from the true domain.  `unnormalize` is called before the true function is evaluated which will first map the points back to the original domain.
# 
# We support either TS and qEI which can be specified via the `acqf` argument.

# In[6]:


def generate_batch(
    state,
    model,  # GP model
    X,  # Evaluated points on the domain [0, 1]^d
    Y,  # Function values
    batch_size,
    n_candidates=None,  # Number of candidates for Thompson sampling
    num_restarts=10,
    raw_samples=512,
    acqf="ts",  # "ei" or "ts"
):
    assert acqf in ("ts", "ei")
    assert X.min() >= 0.0 and X.max() <= 1.0 and torch.all(torch.isfinite(Y))
    if n_candidates is None:
        n_candidates = min(5000, max(2000, 200 * X.shape[-1]))

    # Scale the TR to be proportional to the lengthscales
    x_center = X[Y.argmax(), :].clone()
    weights = gp_model.covar_module.base_kernel.lengthscale.squeeze().detach()
    weights = weights / weights.mean()
    weights = weights / torch.prod(weights.pow(1.0 / len(weights)))
    tr_lb = torch.clamp(x_center - weights * state.length / 2.0, 0.0, 1.0)
    tr_ub = torch.clamp(x_center + weights * state.length / 2.0, 0.0, 1.0)

    if acqf == "ts":
        dim = X.shape[-1]
        sobol = SobolEngine(dim, scramble=True)
        pert = sobol.draw(n_candidates).to(dtype=dtype, device=device)
        pert = tr_lb + (tr_ub - tr_lb) * pert

        # Create a perturbation mask
        prob_perturb = min(20.0 / dim, 1.0)
        mask = torch.rand(n_candidates, dim, dtype=dtype, device=device) <= prob_perturb
        ind = torch.where(mask.sum(dim=1) == 0)[0]
        mask[ind, torch.randint(0, dim - 1, size=(len(ind),), device=device)] = 1

        # Create candidate points from the perturbations and the mask
        X_cand = x_center.expand(n_candidates, dim).clone()
        X_cand[mask] = pert[mask]

        # Sample on the candidate points
        thompson_sampling = MaxPosteriorSampling(model=gp_model, replacement=False)
        with torch.no_grad():  # We don't need gradients when using TS
            X_next = thompson_sampling(X_cand, num_samples=batch_size)

    elif acqf == "ei":
        ei = qExpectedImprovement(gp_model, train_Y.max())
        X_next, acq_value = optimize_acqf(
            ei,
            bounds=torch.stack([tr_lb, tr_ub]),
            q=batch_size,
            num_restarts=num_restarts,
            raw_samples=raw_samples,
        )

    return X_next


# ## Optimization loop
# This simple loop runs one instance of TuRBO-1 with Thompson sampling until convergence.
# 
# TuRBO-1 is a local optimizer that can be used for a fixed evaluation budget in a multi-start fashion.  Once TuRBO converges, `state["restart_triggered"]` will be set to true and the run should be aborted.  If you want to run more evaluations with TuRBO, you simply generate a new set of initial points and then keep generating batches until convergence or when the evaluation budget has been exceeded.  It's important to note that evaluations from previous instances are discarded when TuRBO restarts.
# 
# NOTE: We use a `SingleTaskGP` with a noise constraint to keep the noise from getting too large as the problem is noise-free. 

# In[7]:


X_turbo = get_initial_points(dim, n_init)
Y_turbo = torch.tensor(
    [eval_objective(x) for x in X_turbo], dtype=dtype, device=device
).unsqueeze(-1)

state = TurboState(dim, batch_size=batch_size, best_value=max(Y_turbo).item())

NUM_RESTARTS = 10 #if not SMOKE_TEST else 2
RAW_SAMPLES = 512 #if not SMOKE_TEST else 4
N_CANDIDATES = 5000 #min(5000, max(2000, 200 * dim)) if not SMOKE_TEST else 4

torch.manual_seed(0)

while not state.restart_triggered:  # Run until TuRBO converges
    # Fit a GP model
    train_Y = (Y_turbo - Y_turbo.mean()) / Y_turbo.std()
    likelihood = GaussianLikelihood(noise_constraint=Interval(1e-8, 1e-3))
    covar_module = ScaleKernel(  # Use the same lengthscale prior as in the TuRBO paper
        MaternKernel(
            nu=2.5, ard_num_dims=dim, lengthscale_constraint=Interval(0.005, 4.0)
        )
    )
    gp_model = SingleTaskGP(
        X_turbo, train_Y, covar_module=covar_module, likelihood=likelihood
    )
    mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)

    # Do the fitting and acquisition function optimization inside the Cholesky context
    with gpytorch.settings.max_cholesky_size(max_cholesky_size):
        # Fit the gp_model
        fit_gpytorch_mll(mll)

        # Create a batch
        X_next = generate_batch(
            state=state,
            model=gp_model,
            X=X_turbo,
            Y=train_Y,
            batch_size=batch_size,
            n_candidates=N_CANDIDATES,
            num_restarts=NUM_RESTARTS,
            raw_samples=RAW_SAMPLES,
            acqf="ts",
        )

    Y_next = torch.tensor(
        [eval_objective(x) for x in X_next], dtype=dtype, device=device
    ).unsqueeze(-1)

    # Update state
    state = update_state(state=state, Y_next=Y_next)

    # Append data
    X_turbo = torch.cat((X_turbo, X_next), dim=0)
    Y_turbo = torch.cat((Y_turbo, Y_next), dim=0)

    # Print current status
    print(
        f"{len(X_turbo)}) Best value: {state.best_value:.2e}, TR length: {state.length:.2e}"
    )


In [ ]:
loss_min_params, loss, loss_min_list = optimize_pso(model, observed_rays, uncompensated_parameters_selected, iterations=2000, num_candidates=1000)

In [ ]:
loss_min_params, loss, loss_min_list = optimize_ea(model, observed_rays, uncompensated_parameters_selected, iterations=1000, num_candidates=1000)

In [ ]:
loss_min_params, loss, loss_min_list = optimize_brute(model, observed_rays, uncompensated_parameters_selected, iterations=1000, num_candidates=10000)

In [ ]:
loss_min_params, loss, loss_min_list = optimize_smart_walker(model, observed_rays, uncompensated_parameters_selected, iterations = 1000, num_candidates=10000)

In [ ]:
#fig = plot_param_tensors(loss_min_params[[1,2,4,8]], uncompensated_parameters_selected[[1,2,4,8]], engine = engine, ray_parameter_container=model.input_parameter_container, compensated_parameters=compensated_parameters_selected[[1,2,4,8]])

method_dict = {"smart walker": (optimize_smart_walker, 1000), "brute": (optimize_brute, 1000), "pso": (optimize_pso, 1000), "ea": (optimize_ea, 1000), "evo": (optimize_evotorch, 1000)}
method_evaluation_dict = {}
method_dict = {"ea": (optimize_ea, 1000), "evo": (optimize_evotorch, 1000)}

for key, entry in tqdm(method_dict.items(), desc="Evaluating methods"):
    loss_best, mean_progress, std_progress, loss_min_params_tens, offset_rmse = evaluate_evaluation_method(entry[0], model, repetitions=2, num_candidates=entry[1], iterations=10)
    method_evaluation_dict[key]= (loss_best, mean_progress, std_progress, offset_rmse)

def run_optimizer(optimizer, model, observed_rays, uncompensated_parameters, iterations, num_candidates):
    return optimizer(model, observed_rays, uncompensated_parameters, iterations, num_candidates)

repetitions= 2
iterations = 20
for key, (optimizer, num_candidates) in method_dict.items():
    t0 = benchmark.Timer(
        stmt='run_optimizer(optimizer, model, observed_rays, uncompensated_parameters_selected, iterations=iterations, num_candidates=num_candidates)',
        setup='from __main__ import run_optimizer',
        globals={'optimizer': optimizer, 'model': model, 'observed_rays': observed_rays, 'iterations': iterations, 'uncompensated_parameters_selected': uncompensated_parameters_selected, 'num_candidates': num_candidates},
        num_threads=1,
        label='optimize '+key,
        sub_label=None)
    timing_results = t0.blocked_autorange()
    timings = torch.tensor(timing_results.times)
    execution_time = torch.min(timings).item()
    method_evaluation_dict[key] += (execution_time,)

In [ ]:
plot_optimizer_iterations(method_evaluation_dict, outputs_dir='../../outputs')

In [ ]:
t0 = benchmark.Timer(
    stmt='optimize_smart_walker(model, observed_rays, uncompensated_parameters_selected)',
    setup='from __main__ import optimize_smart_walker',
    globals={'model': model, 'observed_rays': observed_rays, 'uncompensated_parameters_selected': uncompensated_parameters_selected},
    num_threads=1,
    label='optimize smart walker',
    sub_label='optimize smart walker')
print(t0.timeit(repetitions))

In [ ]:
plot_observed_rays = observed_rays.squeeze()
plot_min_param_rays = model(loss_min_params)
fig, ax = plt.subplots(1, plot_observed_rays.shape[0], sharex=True, sharey=True, figsize=(32, 9))
with torch.no_grad():
    for i in range(plot_observed_rays.shape[0]):
        ax[i].plot(model(loss_min_params)[i].cpu())
        ax[i].plot(observed_rays.squeeze()[i].cpu())

In [ ]:
plot_param_tensors(loss_min_params[:,0], uncompensated_parameters_selected[:,0], engine = engine, ray_parameter_container=model.input_parameter_container, compensated_parameters=compensated_parameters_selected[:,0], )

In [ ]:
fig = fancy_plot_param_tensors(loss_min_params[:], uncompensated_parameters_selected[:].squeeze(), engine = engine, ray_parameter_container=model.input_parameter_container, compensated_parameters=compensated_parameters_selected[:].squeeze())
import plotly.io as pio

# Save the figure to an HTML file
pio.write_html(fig, 'figure.html')

In [ ]:
from ray_tools.base.transform import Histogram, RayTransformConcat, XYHistogram

transforms = [
        XYHistogram(50, (-10., 10.), (-3., 3.))
]

fig, ax = plt.subplots(1,2, sharey=True, squeeze=False)

x_simulation_hist_list = []
y_simulation_hist_list = []
for i in trange(2):
    out = engine.run(tensor_list_to_param_container_list(loss_min_params), XYHistogram(50, (-10., 10.), (-3., 3.)))
    out_simulation = out[-1]['ray_output']['ImagePlane']['0.0']
    x_simulation_hist, _ = torch.histogram(out_simulation.x_loc,bins=50, range=[-10, 10])
    y_simulation_hist, _ = torch.histogram(out_simulation.y_loc,bins=50, range=[-3, 3])
    x_simulation_hist_list.append(x_simulation_hist / 22594.)
    y_simulation_hist_list.append(y_simulation_hist / 22594.)
    
    ax[0, 0].plot(torch.linspace(-10, 10, 50), x_simulation_hist / 22594.)
    ax[0, 1].plot(torch.linspace(-3, 3, 50), y_simulation_hist / 22594.)

In [ ]:
loss_min_params.shape
t = tensor_list_to_param_container_list(loss_min_params)
out = [engine.run(tensor_list_to_param_container_list(loss_min_params), XYHistogram(50, (-10., 10.), (-3., 3.))) for i in trange(20)]

In [ ]:
loss_min_params

In [ ]:
hist_list = []
for i in out:
    hist_list.append(torch.vstack([j['ray_output']['ImagePlane']['histogram'].reshape(-1) / 22594. for j in i]))
#        plt.plot(j['ray_output']['ImagePlane']['histogram'].reshape(-1) / 22594.)

In [ ]:
hist_list_tensor = torch.stack(hist_list)
#print(hist_list_tensor.shape)
print("var",hist_list_tensor.var(dim=0, correction=0).mean().item())

# Datamodule

In [ ]:


#bal_memory_dataset = BalancedMemoryDataset(dataset=dataset, load_len=load_len, min_n_rays=1)
#memory_dataset = MemoryDataset(dataset=dataset, load_len=load_len)
#datamodule = DefaultDataModule(dataset=bal_memory_dataset, num_workers=4)
#datamodule.prepare_data()
#datamodule.setup(stage="test")
#test_dl = datamodule.test_dataloader()

#unbal_datamodule = DefaultDataModule(dataset=memory_dataset, num_workers=4)
#unbal_datamodule.prepare_data()
#unbal_datamodule.setup(stage="test")

#unbal_test_dl = unbal_datamodule.test_dataloader()

In [ ]:
mse_statistics(mses_tensor)

## Maximum distribution

In [ ]:
value_list = []
params_list = []

for i in tqdm(unbal_test_dl):
    biggest = i[1].flatten(start_dim=1)
    biggest, _ = i[1].flatten(start_dim=1).max(dim=1)
    mask = biggest > 0.8
    value_list.append(biggest[mask])
    params_list.append(i[0][mask])
value_tensor = torch.cat(value_list)
params_tensor = torch.cat(params_list)

torch.save(value_tensor, 'outputs/values.pt')
torch.save(params_tensor, 'outputs/params.pt')
plt.hist(value_tensor)
plt.savefig('outputs/max_dist_hist.png')

In [ ]:
value_tensor = torch.load('outputs/values.pt')
params_tensor = torch.load('outputs/params.pt')

In [ ]:
for i in tqdm(test_dl):
    #print(len(i[0][:10]))
    t = tensor_list_to_param_container_list(i[0][:10])
    print(len(t))
    out = [engine.run(t, XYHistogram(50, (-10., 10.), (-3., 3.))) for i in trange(2)]
    break
#['ray_output']['ImagePlane']['histogram']


In [ ]:
print(out[0][0]['ray_output']['ImagePlane']['histogram'])
len(out), len(out[0])

# Special sample

In [ ]:
with open("outputs/special_sample_168_selected.pkl", "rb") as f:
    special_sample = pickle.load(f, fix_imports=True, encoding='ASCII', errors='strict', buffers=None)
observed_params = special_sample.uncompensated_parameters

for param_container in observed_params:
    for label in ['ImagePlane.translationXerror', 'ImagePlane.translationYerror', 'ImagePlane.translationZerror']:
        if label in list(param_container.keys()):
            del param_container[label]

In [ ]:
len(observed_params)
Plot.plot_engines_comparison(engine, surrogate_engine, observed_params[:5], MultiLayer([0.]), )

In [ ]:
uncompensated_parameters = [elem.clone() for elem in special_sample.uncompensated_parameters]
for elem in uncompensated_parameters:
    elem.perturb(special_sample.target_params)
uncompensated_parameters[0]

In [ ]:
Plot.plot_engines_comparison(engine, surrogate_engine, uncompensated_parameters, MultiLayer([0.]))

In [ ]:
mse_comparison, x_simulation_hist, y_simulation_hist = mse_engines_comparison(engine, surrogate_engine, uncompensated_parameters[:5], MultiLayer([0.]))
plt.clf()
fig, ax = plt.subplots(1, 3)
for hist in x_simulation_hist:
    ax[0].plot(hist, alpha=0.3)
for hist in y_simulation_hist:
    ax[1].plot(hist, alpha=0.3)
ax[2].hist(mse_comparison)

In [ ]:
x_loc_list = []
good_param_list = []
batch_size = 5000
for i in trange(15000//batch_size):
    param_container = [tensor_to_param_container(torch.rand((34,))) for _ in range(batch_size)]
    surrogate_out = surrogate_engine.run(param_container, MultiLayer([0.]))
    for j in range(len(param_container)):
        output = surrogate_out[j]['ray_output']['ImagePlane']['xy_hist']
        if output.x_loc.sum() > 0.5:
            x_loc_list.append(output.x_loc.sum())
            good_param_list.append(param_container[j])

observed_containers_tensor = torch.vstack([surrogate_engine.select({"1e5/params":param_container})[0] for param_container in observed_params])
good_containers_tensor = torch.vstack([surrogate_engine.select({"1e5/params":param_container})[0] for param_container in good_param_list])

In [ ]:
plt.clf()
for i in range(good_containers_tensor.shape[0]):
    plt.plot(good_containers_tensor[i], c = 'blue', alpha=0.1)
for i in range(observed_containers_tensor.shape[0]):
    plt.plot(observed_containers_tensor[i], alpha=0.8)
plt.legend(["special", "good"])

In [ ]:
#mask = value_tensor > 0.44
out = ((params_tensor - observed_containers_tensor[0].unsqueeze(0))**2)/2.
out = out.mean(dim=1)
out_sorted, indices = torch.sort(out)
#part_indices = indices[:5]
#print(part_indices.shape)
#min_arg = out.argmin()
#plt.hist(out.mean(dim=1))
#plt.plot(params_tensor[min_arg])
#plt.plot(observed_containers_tensor[0])
for i in indices[:1]:
    plt.plot(params_tensor[i])
Plot.plot_engines_comparison(engine, surrogate_engine, [tensor_to_param_container(params_tensor[min_arg]) for min_arg in indices[:1]], MultiLayer([0.]), )

In [ ]:
transforms = {"ImagePlane": transform for transform in cfg_transforms}
out_engine = engine.run(observed_params, transforms)

In [ ]:
standardized_simulations = surrogate_engine.model.standardizer(torch.vstack([element['ray_output']['ImagePlane']['histogram'].flatten(start_dim=0) for element in out_engine]))
a = ((standardized_simulations - out_model)**2).mean(dim=1)

In [ ]:
plt.hist(a)

# Good params vs. bad params

In [ ]:
params_list = []
num_rays_list = []
for i in tqdm(unbal_test_dl):
    params_list.append(i[0])
    num_rays_list.append(i[1])
params_tensor = torch.vstack(params_list)
num_rays_tensor= torch.vstack(num_rays_list)
plt.hist(torch.tensor(num_rays_list))

In [ ]:
biggest = torch.tensor(num_rays_list).argmax()
test_parameters = memory_dataset[biggest][0]
param_container_list = [tensor_to_param_container(test_parameters)]

fig, ax = plt.subplots(1,2, sharey=True, squeeze=False)

x_simulation_hist_list = []
y_simulation_hist_list = []
for i in trange(20):
    out = engine.run(param_container_list, MultiLayer([0.]))
    out_simulation = out[-1]['ray_output']['ImagePlane']['0.0']
    x_simulation_hist, _ = torch.histogram(out_simulation.x_loc,bins=50, range=[-10, 10])
    y_simulation_hist, _ = torch.histogram(out_simulation.y_loc,bins=50, range=[-3, 3])
    x_simulation_hist_list.append(x_simulation_hist / 22594.)
    y_simulation_hist_list.append(y_simulation_hist / 22594.)
    
    ax[0, 0].plot(torch.linspace(-10, 10, 50), x_simulation_hist / 22594.)
    ax[0, 1].plot(torch.linspace(-3, 3, 50), y_simulation_hist / 22594.)


In [ ]:
x_simulation_hist_tens = torch.vstack(x_simulation_hist_list)
y_simulation_hist_tens = torch.vstack(y_simulation_hist_list)

In [ ]:
x_simulation_hist_tens.var(dim=0).mean()

In [ ]:
x_simulation_hist_tens.mean(dim=0).shape

In [ ]:
params_tensor = hist_dataset.data_dict['histogram/ImagePlane']#['parameters']
num_rays_tensor = hist_dataset.data_dict['n_rays/ImagePlane']
print(params_tensor.shape, num_rays_tensor.shape)

In [ ]:
import umap

mask = num_rays_tensor >= 5.# 100.
data_tensor = params_tensor[mask]
print(data_tensor.shape)
data_tensor = data_tensor[:,:]
class_tensor = num_rays_tensor[mask]

data_np = data_tensor.numpy()
class_np = class_tensor.numpy().flatten()
print(data_np.shape)
umap_model = umap.UMAP(n_neighbors=20, min_dist=0.1, n_components=2)
umap_embedding = umap_model.fit_transform(data_np)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1], c=class_np, cmap='Spectral', s=5)
plt.colorbar(scatter)
plt.title('UMAP projection of the dataset')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.show()

In [ ]:
dt = torch.vstack([model(cp) for cp in compensated_parameters_list]).cpu()
ct = torch.cat([torch.ones((len(compensated_parameters_list[i])*dt.shape[1],)).float() * i for i in range(len(compensated_parameters_list))]).cpu()
#ct = torch.where(ct == 0, 0., 1.)
#dt = dt.flatten(start_dim=0, end_dim=-2)
print(dt.shape, ct.shape)

In [ ]:
import umap
mask = num_rays_tensor >= 1000.# 100.
print(observed_rays_real.shape)
observed_real_tensor = observed_rays_real.flatten(start_dim=0, end_dim=-2).cpu()
observed_sim_tensor = hist_dataset.data_dict['histogram/ImagePlane'][mask] #observed_rays.flatten(start_dim=0, end_dim=-2).cpu()#

observed_real_label = torch.ones((observed_real_tensor.shape[0],)).float()
observed_sim_label = torch.ones((observed_sim_tensor.shape[0],)).float() * 2.

params_tensor
#
data_tensor = torch.cat((observed_real_tensor, observed_sim_tensor))
class_tensor = torch.cat((observed_real_label, observed_sim_label))

data_np = data_tensor.numpy()
class_np = class_tensor.numpy().flatten()
print(data_np.shape)
umap_model = umap.UMAP(n_neighbors=300, min_dist=0.1, n_components=2)
umap_embedding = umap_model.fit_transform(data_np)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1], c=class_np, cmap='Spectral', s=5)
plt.colorbar(scatter)
plt.title('UMAP projection of the dataset')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.show()

In [ ]:
observed_real_tensor = observed_rays_real.flatten(start_dim=0, end_dim=-2).cpu()
observed_sim_tensor = hist_dataset.data_dict['histogram/ImagePlane'][mask] #observed_rays.flatten(start_dim=0, end_dim=-2).cpu()#

observed_real_tensor.shape, observed_sim_tensor.shape

In [ ]:
observed_rays_real.flatten(start_dim=1, end_dim=-1).cpu().shape #.flatten(start_dim=0, end_dim=-2).cpu().shape
dt.flatten(start_dim=1, end_dim=-1).cpu().shape

In [ ]:
def plot_umap(umap_embedding, class_np):
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1], c=class_np, cmap='bwr', s=5)
    plt.colorbar(scatter)
    plt.title('UMAP projection of the dataset')
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.show()

def calculate_umap(observed_real_tensor, observed_sim_tensor, n_neighbors=300, min_dist=0.1):
    observed_real_label = torch.ones((observed_real_tensor.shape[0],)).float()
    observed_sim_label = torch.ones((observed_sim_tensor.shape[0],)).float() * 3.
    
    
    data_tensor = torch.cat((observed_real_tensor, observed_sim_tensor))
    class_tensor = torch.cat((observed_real_label, observed_sim_label))
    print(data_tensor.shape, class_tensor.shape)
    
    data_np = data_tensor.numpy()
    class_np = class_tensor.numpy().flatten()
    
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=2)
    umap_embedding = umap_model.fit_transform(data_np)
    return umap_embedding, class_np

## Generated problems z-included UMAP

In [ ]:
observed_real_tensor = observed_rays_real.flatten(start_dim=1, end_dim=-1).cpu()
observed_sim_tensor = dt.flatten(start_dim=1, end_dim=-1).cpu()

umap_embedding, class_np = calculate_umap(observed_real_tensor, observed_sim_tensor, n_neighbors=2, min_dist=0.01)
plot_umap(umap_embedding, class_np)

## Generated problems all beamlines z-included UMAP

In [ ]:
observed_sim_tensor = torch.stack([model(cp)[:14] for cp in compensated_parameters_list]).cpu().flatten(start_dim=1)
observed_real_tensor = observed_rays_real[:14].unsqueeze(0).cpu().flatten(start_dim=1)

In [ ]:
#observed_sim_tensor.shape, observed_real_tensor.shape
umap_embedding, class_np = calculate_umap(observed_real_tensor, observed_sim_tensor[:5000], n_neighbors=10, min_dist=0.9)
plot_umap(umap_embedding, class_np)

In [ ]:
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=1000)
tsne_embedding = tsne_model.fit_transform(data_np)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(tsne_embedding[:, 0], tsne_embedding[:, 1], c=class_np, cmap='Spectral', s=5)
plt.colorbar(scatter)
plt.title('t-SNE projection of the dataset')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.show()

In [ ]:
import glob
import tqdm
import matplotlib.pyplot as plt
import torch

from ray_nn.nn.xy_hist_data_models import MetrixXYHistSurrogate, StandardizeXYHist
from ray_tools.simulation.torch_datasets import MemoryDataset, HistDataset
from datasets.metrix_simulation.config_ray_emergency_surrogate import PARAM_CONTAINER_FUNC as params
from torch.utils.data import DataLoader
from ray_nn.data.transform import Select

model_path = "../../outputs/xy_hist/ee8cvj82/checkpoints/epoch=36-step=35212012.ckpt"
model = MetrixXYHistSurrogate.load_from_checkpoint(model_path)
model.to(torch.device('cpu'))
model.compile()
model.eval()

load_len: int | None = 1000
h5_files = list(glob.iglob('datasets/metrix_simulation/ray_emergency_surrogate_50+50+z/histogram_*.h5'))
sub_groups = ['parameters', 'histogram/ImagePlane', 'n_rays/ImagePlane']
transforms=[lambda x: x[1:].float(), lambda x: standardizer(x.flatten().float()), lambda x: x.int()]
dataset = HistDataset(h5_files, sub_groups, transforms, normalize_sub_groups=['parameters'], load_max=load_len)


memory_dataset = MemoryDataset(dataset=dataset, load_len=load_len)

train_dataloader = DataLoader(memory_dataset, batch_size=2048, shuffle=False, num_workers=0)

errors_list = []
with torch.no_grad():
    for par_input, label, _ in tqdm(train_dataloader):
        out = model(par_input)
        label = label.flatten(start_dim=1)
        b = ((label - out)**2).mean(dim=1)
        errors_list.append(b)
errors_tensor = torch.cat(errors_list)

plt.hist(errors_tensor)
plt.savefig('outputs/dataset_errors_hist.png')
torch.save(errors_tensor, 'outputs/dataset_errors.pt')

# Import real data

In [ ]:
from sub_projects.ray_optimization.real_data import import_data #, 
#[-15., -10., -5., 0., 5., 10., 15., 20., 25., 30.]
def import_real_hist_data(parameter_container, device, path = '../../datasets/metrix_real_data/2021_march_complete', import_set = ['M10', 'M18', 'M22', 'M23', 'M24', 'M25', 'M27', 'M28', 'M29', 'M30', 'M32', 'M33', 'M36',
                             'M37', 'M40', 'M41', 'M42', 'M43', 'M44'], z_layers=[-15., -10., -5., 0., 5., 10., 15., 20., 25., 30.], check_value_lims=False, z_array_label='ImagePlane.translationZerror'):
    imported_data = import_data(
                path,
                import_set,
                z_layers,
                parameter_container,
                check_value_lims=check_value_lims,
            )
    xy_hist = XYHistogram(50, (-10., 10.), (-3., 3.))
    z_array_min, z_array_max = model.input_parameter_container[z_array_label].value_lims
    normalized_z_array = torch.tensor((z_layers - z_array_min) / (z_array_max - z_array_min), device=device).float()

    # observed_rays_point_cloud
    real_data_point_cloud_list = []
    for i in range(len(imported_data)):
        real_data_point_cloud_list.append(imported_data[i])
    observed_rays_point_cloud = [ray_dict_to_tensor(entry, 'ImagePlane') for entry in real_data_point_cloud_list]

    real_data_list = []
    for i in range(len(imported_data)):
        real_data_list.append(xy_hist(imported_data[i]['ray_output']['ImagePlane']))
    z_layer_real_data_tensor_list = []
    for z in z_layers:
        z_layer_real_data_tensor_list.append(torch.stack([real_data_list[i][z]['histogram'] for i in range(len(real_data_list))]))
    real_data_tensor = torch.stack(z_layer_real_data_tensor_list, dim=1)

    real_data_tensor_normalized = real_data_tensor / 4000. #surrogate_engine.model.standardizer(real_data_tensor)
    observed_rays = real_data_tensor_normalized.flatten(start_dim=-2).unsqueeze(2).float().to(device)

    uncompensated_parameters_list = []
    for i in range(len(imported_data)):
        uncompensated_entry = torch.tensor([value.get_value() for value in Plot.normalize_parameters(imported_data[i]['param_container_dict'], parameter_container).values()])
        uncompensated_parameters_list.append(uncompensated_entry)
    uncompensated_parameters = torch.stack(uncompensated_parameters_list)
    uncompensated_parameters = uncompensated_parameters.unsqueeze(1).float().to(device)
    uncompensated_parameters = uncompensated_parameters.repeat_interleave(len(z_layers), dim=1).unsqueeze(2)
    uncompensated_parameters[:, :, 0, -1] = normalized_z_array
    
    #print(uncompensated_parameters)
    
    return observed_rays, uncompensated_parameters, observed_rays_point_cloud
    
#observed_rays_real, uncompensated_parameters_real, observed_rays_point_cloud = import_real_hist_data(model.input_parameter_container, model.device, path = '../../datasets/metrix_real_data/2021_march_complete', import_set = ['M03'], z_layers=[0., 5., 10.], check_value_lims=False, z_array_label='ImagePlane.translationZerror')
#plt.clf()
#plt.plot(observed_rays_real[0][0][0].cpu())
#plt.show()   
observed_rays_real, uncompensated_parameters_real, observed_rays_point_cloud = import_real_hist_data(model.input_parameter_container, device=model.device)

In [ ]:
seed = 1000042
loss_min_params, loss, loss_min_list = optimize_evotorch_ga(model, observed_rays_real, uncompensated_parameters_real, iterations=1000, num_candidates=500, mutation_scale=0.2, sbx_crossover_rate=0.8, tournament_size=3, seed=seed)
compensated_rays = model(loss_min_params.clone())
fig = MetrixXYHistSurrogate.plot_data_3(compensated_rays[:3, 0].cpu(), observed_rays_real[:3, 0,0].cpu(), label_list=["Compensated", "Experiment"])
plt.savefig('../../outputs/real_world_optimized.pdf', bbox_inches='tight')

In [ ]:
loss_min_params, loss, loss_min_list  = optimize_smart_walker(model, observed_rays_real, uncompensated_parameters_real, iterations=1000, num_candidates=40000)

In [ ]:
#torch.set_default_device('cuda')
print(loss_min_params.shape)
pc = [tensor_to_param_container(loss_min_params[i][0].cpu(), ray_parameter_container=model.input_parameter_container) for i in range(loss_min_params.shape[0])]
Plot.plot_engines_comparison(engine, surrogate_engine, pc[:8], MultiLayer([0.]), )

In [ ]:
def plot_engines_comparison_2(input_param_tensor, engine, model):
    engine_outputs = simulate_param_tensor(input_param_tensor, engine, ray_parameter_container=model.input_parameter_container, exported_plane='ImagePlane')
    std_backward = model.standardizer.destandardize
    model_outputs = std_backward(model(input_param_tensor)).detach().cpu()

    fig, ax = plt.subplots(len(engine_outputs),2 * model_outputs[0].shape[0], sharey=True, squeeze=False, figsize=(14,5))
    for i in range(len(engine_outputs)):
        for j in range(model_outputs[0].shape[0]):
            model_hist = model_outputs[i, j]
            out_simulation = engine_outputs[i][j]

            intensity = (model.intensity_multiplier * input_param_tensor[i, j, -4] + model.intensity_addend).unsqueeze(-1).cpu()
            #print(input_param_tensor[i, j, -4])
            x_simulation_hist, _ = torch.histogram(out_simulation[:, 0],bins=50, range=[-10, 10])
            line1, = ax[i, 2*j].plot(torch.linspace(-10, 10, 50).cpu(), x_simulation_hist.cpu()*intensity, label="simulation")
            line2, = ax[i, 2*j].plot(torch.linspace(-10, 10, 50).cpu(), model_hist[:50], label="surrogate")
            y_simulation_hist, _ = torch.histogram(out_simulation[:, 1],bins=50, range=[-3, 3]) 
            ax[i, 2*j+1].plot(torch.linspace(-3, 3, 50).cpu(), y_simulation_hist.cpu()*intensity)
            ax[i, 2*j+1].plot(torch.linspace(-3, 3, 50).cpu(), model_hist[50:])
            ax[0, 0].legend(handles=[line1, line2])
        
    return fig

plot_engines_comparison_2(loss_min_params[:10, :1], engine, model)

In [ ]:
loss_min_params.shape, loss_min_params[:, 0, :]

In [ ]:
fig = fancy_plot_param_tensors(loss_min_params[:], uncompensated_parameters_real[:].squeeze(2), engine = engine, ray_parameter_container=model.input_parameter_container, observed_rays_point_cloud=observed_rays_point_cloud)
pio.write_html(fig, os.path.join('fancy.html'))

In [ ]:
plt.clf()
print(observed_rays_real.shape)
fig, ax = plt.subplots(5)
for i in range(5):
    for j in range(3):
#ax[0].plot(model(loss_min_params[0])[0].cpu())
       ax[i].plot(observed_rays_real[i][j][0].cpu())
plt.show()

In [ ]:
uncompensated_parameters_real[:, :1].shape

In [ ]:
plot_param_tensors(loss_min_params[:,:1], uncompensated_parameters_real[:, :1].squeeze(1), engine=engine, ray_parameter_container=model.input_parameter_container, observed_rays_point_cloud=observed_rays_point_cloud)

In [ ]:
plot_param_tensors(loss_min_params[:], uncompensated_parameters_selected[:], compensated_parameters=compensated_parameters_selected[:])

In [ ]:
repeated_params = torch.vstack(10*[loss_min_params[1]])
a = repeated_params.clone()
b = repeated_params.clone()
c = repeated_params.clone()
for i, ten in enumerate(a):
    ten[-1]=0.5+0.1*i
    ten[-2]=0.5
for i, ten in enumerate(b):
    ten[-2]=0.5+0.1*i
    ten[-1]=0.5
for i, ten in enumerate(c):
    ten[-1]=0.5
    ten[-2]=0.5#+0.005*i
plot_param_tensors(a[:3], b[:3], compensated_parameters=c[:3])
#print(repeated_params)
#plot_param_tensors(loss_min_params[:], uncompensated_parameters_selected[:], compensated_parameters=compensated_parameters_selected[:])

In [ ]:
loss_min_params.shape, uncompensated_parameters_selected.shape, compensated_parameters_selected.shape

In [ ]:
parameter_comparison_plot = Plot.plot_param_comparison(
    predicted_params=tensor_list_to_param_container_list(loss_min_params.squeeze().unsqueeze(0)),
    epoch=42,
    training_samples_count=len(observed_rays),
    search_space=model.offset_space,
    real_params=tensor_list_to_param_container_list(compensated_parameters_selected)[0],
)

In [ ]:
parameter_comparison_plot = Plot.plot_param_comparison(
    predicted_params=tensor_to_param_container(offsets_selected.squeeze()),
    epoch=42,
    training_samples_count=len(observed_rays),
    search_space=RayOptimization.limited_search_space(model.offset_space, RandomGenerator(42), max_deviation=max_offset),
    real_params=tensor_list_to_param_container_list(loss_min_params[0] - compensated_parameters_selected.squeeze())[1],
)

In [ ]:
predicted_params = (loss_min_params - compensated_parameters_selected.squeeze())[0][0]
real_params = offsets_selected[0].cpu()
Plot.plot_normalized_param_comparison(
    predicted_params=model.unscale_offset(predicted_params).cpu(),
    labels= [key for key, value in model.input_parameter_container.items() if isinstance(value, MutableParameter)],
    real_params=real_params,
)
plt.savefig('test.pdf', bbox_inches='tight')

In [ ]:

loss_min_params.shape, uncompensated_parameters_selected.squeeze(-2).shape
offsets_predicted = (loss_min_params-uncompensated_parameters_selected.squeeze(-2))[0][0]
mutable_keys = [key for key, value in model.input_parameter_container.items() if isinstance(value, MutableParameter)]
offsets_predicted_normalized = model.unscale_offset(offsets_predicted)
offsets_predicted_normalized.min(), offsets_predicted_normalized.max()
plot_param_comparison(
    predicted_params=offsets_predicted_normalized.cpu(),
    labels= mutable_keys,
    real_params=offsets_selected[0].cpu()
)

# SGD

In [ ]:
batch_size = 10000
x = torch.rand((batch_size, uncompensated_parameters.shape[-1]), requires_grad=True, device=model.device)
optimize_algo = 'adam'
if optimize_algo == 'adam':
    optimizer = torch.optim.Adam([x], lr=0.01)  # You can adjust the learning rate as needed
else:
    optimizer = torch.optim.SGD([x], lr=0.01)  # Adjust the learning rate as needed
torch.autograd.set_detect_anomaly(True)
num_epochs = 1000  # Number of iterations
uncompensated_parameters = uncompensated_parameters_selected
#with torch.no_grad():
observed_rays = model(compensated_parameters_selected, clone_output=True, grad=True)

def function_to_minimize(x):
    tensor_sum = x + uncompensated_parameters
    compensated_rays = model(tensor_sum, clone_output=True, grad=True)
    compensated_rays = compensated_rays.flatten(start_dim=2)
    loss_orig = ((compensated_rays - observed_rays) ** 2).mean(0).mean(-1)
    return loss_orig

pbar = trange(num_epochs)
for epoch in pbar:
    optimizer.zero_grad()  # Clear the gradients from the previous step
    
    output = function_to_minimize(x.detach())  # Compute the function's output
    loss = output.mean()  # Compute the mean loss for this batch
    
    loss.backward(retain_graph=True)  # Backpropagate to compute the gradients
    optimizer.step()  # Update the parameters with SGD

    # Optionally print the loss to monitor progress
    if epoch % 100 == 0:
        pbar.set_postfix({"loss": loss.item()})

In [ ]:
import torch
a = torch.randn(100, 2)
b= torch.randn(100, 2)
loss = torch.nn.MSELoss(reduction="none")

In [ ]:
loss(a,b).mean()